In [1]:
import os
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import urllib


# Red Neuronal convolucional compleja - dataset cifar 10

## Precaucion: 
Se debe correr en una computadora potente, ya que la rutina es muy demandante, se puede jugar con los valores
 batck_size, generations y eval_every, para hacer menos demandante el código.

In [2]:
batch_size = 128
output_every = 50
generations = 200000
eval_every = 100
image_height = 32
image_width = 32
crop_height = 24
crop_width = 24
num_channels = 3
num_targets = 10

data_folder = "cifar-10-batches-bin"

learning_rate = 0.1
lr_decay = 0.9
num_gens_to_wait = 250

image_vect_length = image_height * image_width * num_channels
record_length = 1 + image_vect_length

data_dir = '../datasets/cifar-10-temp'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
cifar10_url = "http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz"
data_file = os.path.join(data_dir, 'cifar-10-binary.tar.gz')
if not os.path.isfile(data_file):
    filepath, _ = urllib.request.urlretrieve(cifar10_url, data_file)
    tarfile.open(filepath, 'r:gz').extractall(data_dir)

In [3]:
def read_cifar_files(record_bytes, distort_images=True):
    image_label = tf.cast(record_bytes[0], tf.int32)

    image = tf.reshape(record_bytes[1:], [num_channels, image_height, image_width])
    reshaped_image = tf.transpose(image, [1, 2, 0])
    reshaped_image = tf.cast(reshaped_image, tf.float32)

    final_image = tf.image.resize_with_crop_or_pad(reshaped_image, crop_width, crop_height)

    if distort_images:
        final_image = tf.image.random_flip_left_right(final_image)
        final_image = tf.image.random_brightness(final_image, max_delta=63)
        final_image = tf.image.random_contrast(final_image, lower=0.2, upper=1.8)

    final_image = tf.image.per_image_standardization(final_image)
    return final_image, image_label

def input_pipeline(batch_size, train_logical=True):
    if train_logical:
        files = [os.path.join(data_dir, data_folder, f'data_batch_{i}.bin') for i in range(1, 6)]
    else:
        files = [os.path.join(data_dir, data_folder, 'test_batch.bin')]

    def parse_record(record):
        record_bytes = tf.io.decode_raw(record, tf.uint8)
        return read_cifar_files(record_bytes, distort_images=train_logical)

    dataset = tf.data.FixedLengthRecordDataset(files, record_length)
    dataset = dataset.map(parse_record)
    dataset = dataset.shuffle(1000).batch(batch_size)

    return dataset

def cifar_cnn_model():
    model = tf.keras.Sequential([
        #tf.keras.layers.Input(shape=(image_width, image_height, num_channels)),  # Definir la entrada explícitamente
        tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu', input_shape=(crop_height, crop_width, num_channels)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(384, activation='relu'),
        tf.keras.layers.Dense(192, activation='relu'),
        tf.keras.layers.Dense(num_targets)
    ])

    return model

# Prepare datasets
train_dataset = input_pipeline(batch_size, train_logical=True)
test_dataset = input_pipeline(batch_size, train_logical=False)

# Create model
model = cifar_cnn_model()

# Compile model
#model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, decay=lr_decay, momentum=0.9),
#              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#              metrics=['accuracy'])


# Compilación del modelo
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, weight_decay=lr_decay, momentum=0.9),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # Usar from_logits=True si las salidas no tienen softmax
    metrics=['accuracy']
)

# Opcional: ver la estructura del modelo para asegurarte de que sea correcta
model.summary()

c:\Users\jrgui\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 24, 24, 64)     │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12, 12, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 12, 12, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 6, 6, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 384)            │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 192)            │        73,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,930 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,068,810 (4.08 MB)

 Trainable params: 1,068,554 (4.08 MB)

 Non-trainable params: 256 (1.00 KB)

In [8]:
# Training
train_loss = []
test_acc = []
a = 0

for i in range(generations):
    for batch in train_dataset:
        images, targets = batch
        loss_value = model.train_on_batch(images, targets)
    
    if i == int:
        a += 1

    if a % 5000 == 0:
        print("5000 mas")   

    if (i + 1) % output_every == 0:
        train_loss.append(loss_value)
        print(f"Paso {i+1}, Pérdida: {loss_value:.5f}")

    if (i + 1) % eval_every == 0:
        for batch in test_dataset:
            test_images, test_targets = batch
            acc = model.evaluate(test_images, test_targets, verbose=0)[1]
            test_acc.append(acc)
        print(f"--- Precisión en test: {acc*100:.2f}% ---")




In [ ]:
# Plot results
eval_idx = range(0, generations, eval_every)
output_idx = range(0, generations, output_every)

#Grafica del error
plt.plot(output_idx, train_loss, 'k-')
plt.title("Softmax Loss para cada iteración")
plt.xlabel("Iteración")
plt.ylabel("Pérdida en Softmax")
plt.show()

#Grafica de la presicion
plt.plot(eval_idx, test_acc, 'k-')
plt.title("Mejora de la precisión en cada iteración")
plt.xlabel("Iteración")
plt.ylabel("Precisión")
plt.ylim([0, 1])
plt.show()